### Part 4

You should now evaluate your models on the FakeNews and the LIAR dataset. Arrange all these results in a table to facilitate a comparison between them. You should be evaluating the model on how well it classifies articles correctly using F-score. You may want to include a confusion matrix to visualize the types of classification errors made by your models.

### Task 1
Evaluate the performance of your Simple and Advanced Models on your FakeNewsCorpus test set. It should be possible to achieve > 80% accuracy but you will not fail the project if your model cannot reach this performance.



In [13]:
# Import the necessary libraries
# pip install dask-ml
import dask.dataframe as dd
import dask_ml.model_selection
import numpy as np

In [14]:
# Read the CSV file into a Dask DataFrame with specified data types
# These dtypes can be found by printing the dask dataframe
cleaned_data = dd.read_csv('data_3GB.csv', encoding="utf-8", dtype={
        'Unnamed: 0': 'object',
        'id': 'object',
        'domain': 'object',
        'type': 'object',
        'url': 'object',
        'content': 'object',
        'scraped_at': 'object',
        'inserted_at': 'object',
        'updated_at': 'object',
        'title': 'object',
        'authors': 'object',
        'keywords': 'float64',
        'meta_keywords': 'object',
        'meta_description': 'object',
        'tags': 'object',
        'summary': 'float64',
        'tokens': 'object',
        'filtered_tokens': 'object',
        'stemmed_tokens': 'object',
    },)

In [15]:
# Define a function to modify the 'type' column values
# This function is used to simplify the classification problem
# The 'reliable' and 'political' types are combined into a single 'reliable' type
def modify_type(x):
    if x == 'reliabl' or x == 'polit':
        return 'reliable'
    else:
        return 'fake'

In [16]:
# Apply the modify_type function to the 'type' column using the map function
cleaned_data['type'] = cleaned_data['type'].map(modify_type, meta=('type', 'object'))

In [17]:
# Identify if there are any missing values in the 'type' column
# If so, they are replaced with an empty string
def nan_to_empty(x):
    if isinstance(x, float) and np.isnan(x):
        return ''
    else:
        return x

In [18]:
# Apply the nan_to_empty function to the 'type' column using the map function
cleaned_data['content'] = cleaned_data['content'].map(nan_to_empty, meta=('content', 'object'))

In [19]:
# Define X and y as the 'content' and 'type' columns
y = cleaned_data['type']
X = cleaned_data['content']

In [20]:
# Split the data into train and test sets
# In an 80/10/10 split

X_train, X_test, y_train, y_test = dask_ml.model_selection.train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)
X_train, X_val, y_train, y_val = dask_ml.model_selection.train_test_split(X_train, y_train, test_size=0.125, random_state=0, shuffle=False)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer object for use in feature extraction
vectorization = TfidfVectorizer()

# Fit the vectorizer to the training data and transform the training data into a vector
# As well as the validation data
xv_train = vectorization.fit_transform(X_train)
xv_test = vectorization.transform(X_test) 

In [22]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1500, class_weight = 'balanced', random_state = 0, C=100)

In [24]:
model.fit(xv_train, y_train)

/Users/Agnes/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=100, class_weight='balanced', max_iter=1500,
                   random_state=0)

In [25]:
# Print the accuracy of the model on the validation data
model.score(xv_test,y_test)

0.8993657658967416

In [26]:
# Predict the class of the validation data
pred_model = model.predict(xv_test)

In [28]:
from sklearn.metrics import classification_report

# Print the classification report for the model
print(classification_report(y_test,pred_model))

              precision    recall  f1-score   support

        fake       0.96      0.91      0.93    213848
    reliable       0.73      0.88      0.80     61287

    accuracy                           0.90    275135
   macro avg       0.84      0.89      0.86    275135
weighted avg       0.91      0.90      0.90    275135



### Task 2
In order to allow you to play around cross-domain performance, try the same exercise on the LIAR dataset, where you know the labels, and can thus immediately calculate the performance. You are expected to directly evaluate the model you trained on the FakeNewsCorpus. In other words, you do not need to retrain the model on the LIAR dataset.

### Task 3
Compare the results of this experiment to the results you obtained in question 3. Report your LIAR results as part of your report. Remember to test the performance of your Simple Model as well.